# Tutoriel Python 3 - Azure ML Studio - MDSF 2018

Azure ML Studio: https://studio.azureml.net

Ce tutoriel a pour but de guider les personnes souhaitant utiliser Python 3 sur [Azure ML Studio](https://studio.azureml.net) pour participer au challenge.

Il comporte 5 étapes :

1. Import des données
2. Analyse descriptive
3. Préparation des données
4. Création d’un modèle
5. Calcul des prédictions et soumissions

# Import des données

Avant de rentrer dans le vif du sujet, installons les packages nécessaires pour ce tutoriel :

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from azureml import Workspace
%matplotlib inline 
pd.set_option('display.max_columns', 500)

In [ ]:
%%time
ws = Workspace()
X_train = pd.read_csv(ws.datasets["X_train.csv"].open(), index_col=0, error_bad_lines=False)
X_test = pd.read_csv(ws.datasets["X_test.csv"].open(), index_col=0, error_bad_lines=False)
y_train = pd.read_csv(ws.datasets["y_train.csv"].open(), index_col=0)

In [ ]:
print("Dimension X_train:", X_train.shape)
print("Dimension X_test:", X_test.shape)

In [ ]:
X_train.head(3)

# Analyse descriptive

## Structure des datasets

Le dataset train comporte les caractéristiques et délai de vente de **8880 objets** vendus sur le site Emmaus. C’est ce dataset que nous allons utiliser pour créer un modèle. Chaque objet est décrit par une observation de X variables. Ces variables sont décrites dans le fichier ```description.pdf``` présent dans la clef USB.

Le dataset test comporte les caractéristiques des **2960 objets** dont il faut prédire le délai de vente. A la différence du train, le délai de vente n’est bien sûr pas renseigné et une colonne ```id``` a ete rajoutée pour identifier les prédictions pendant l’étape de soumission.

In [ ]:
X_train.describe(include='all').T

In [ ]:
y_train.delai_vente.value_counts()

Le jeu de données est très équilibré, chacune des 3 classes a une fréquence proche d’1/3.

# Création d'un modèle

Il est maintenant temps de créer un modele. Dans ce tutoriel nous allons construire une [Forêt Aléatoire](https://fr.wikipedia.org/wiki/For%C3%AAt_d'arbres_d%C3%A9cisionnels)

Pour ce faire nous utilisons les variables ```["poids", "prix", "nb_images", "longueur_image", "largeur_image", "categorie"]```.

Pour éviter le [surapprentissage](https://fr.wikipedia.org/wiki/Surapprentissage) et estimer les vraies performances de notre modèle nous allons utiliser le critère de [validation croisee](https://fr.wikipedia.org/wiki/Validation_crois%C3%A9e) méthode **k-fold** (cross-validation).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder

### Imputation des valeurs manquantes par la valeur "missing"

In [ ]:
X_train.categorie.fillna('missing', inplace=True)
X_test.categorie.fillna('missing', inplace=True)

### Encodage des features catégorielles

Les algorithmes de machine learning s'attendent à avoir en entrée des **nombres**, et non pas des chaînes de caractères. C'est pourquoi nous transformons les **features catégorielles** en nombres, à l'aide de ```LabelEncoder()```

In [ ]:
X_train.categorie.unique()

In [ ]:
le = LabelEncoder()
X_train['categorie'] = le.fit_transform(X_train.categorie)
X_test['categorie'] = le.transform(X_test.categorie)

In [ ]:
features = ["poids", "prix", "nb_images",
            "longueur_image", "largeur_image", "categorie"]

ppl = Pipeline([("imputer", Imputer(strategy='median')),
                ("clf", RandomForestClassifier(n_estimators=10))])

ppl.fit(X_train.loc[:, features], np.ravel(y_train))

pred_train = ppl.predict_proba(X_train.loc[:, features])
pred_cv = cross_val_predict(ppl, X_train.loc[:, features], np.ravel(y_train),
                            method='predict_proba', cv=5, n_jobs=-1)

# Calcul de l'erreur: logloss

In [ ]:
from sklearn.metrics import log_loss 

In [ ]:
print("LogLoss sur echantillon train:",log_loss(y_pred=pred_train, y_true=y_train))
print("LogLoss sur echantillon train (CV):",log_loss(y_pred=pred_cv, y_true=y_train))

# Calcul des predictions

In [ ]:
pred_test = ppl.predict_proba(X_test.loc[:, features])

In [ ]:
df_submission = pd.DataFrame(pred_test, index=X_test.index)

# Soumission

## Possibilité n°1 : via l'API de QScore

1. Aller sur la plateforme [QScore](https://qscore.meilleurdatascientistdefrance.com) puis dans "Submissions" > "Submit from your Python Notebook"
2. Récuperer votre TOKEN
3. Le remplacer dans la fonction ci-dessous et l'exécuter

In [ ]:
import io, math, requests

# Ne fonctionne qu'en Python3, voir commentaire ci-dessous pour Python2
def submit_prediction(df, sep=',', **kwargs):
    # TOKEN a recuperer sur la plateforme: "Submissions" > "Submit from your Python Notebook"
    TOKEN='<TON_TOKEN>'  
    URL='https://qscore.meilleurdatascientistdefrance.com/api/submissions'
    #buffer = io.BytesIO() # Python 2
    buffer = io.StringIO() # Python 3
    df.to_csv(buffer, sep=sep, **kwargs)
    buffer.seek(0)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': buffer})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)

In [ ]:
submit_prediction(df_submission, sep=',', index=True)

## Possibilité n°2 : Soumettez un fichier CSV

1. Aller sur la plateforme [QScore](https://qscore.meilleurdatascientistdefrance.com) puis dans "Submissions" > "Submit with a file"
2. Déposer le fichier CSV

In [ ]:
df_submission.to_csv("my_prediction.csv", index_label="id", header=['0', '1', '2'])